In [6]:
## input data provenant de l'excel
input_path = r"C:\Users\jl0331el\OneDrive - EDF\XMind\output_excel\exported_data_cetac.xlsx"

## sauvegarder le fichier .mm
output_path = r"C:\Users\jl0331el\OneDrive - EDF\XMind\output_xml/output_cetac.mm"

In [7]:
import pandas as pd
import xml.etree.ElementTree as ET


def create_node(parent, text, link=None, color=None, style=None):
    """
    créer un noeud pour freeMind, ajouter url
    """
    attributes = {"TEXT": text}
    if link:
        attributes["LINK"] = link  # considérer URL comme un attribut
    if color:
        attributes["COLOR"] = color  # paramétrer la couleur des noeuds
    if style:
        attributes["STYLE"] = style  # paramétrer le style des liens
    node = ET.SubElement(parent, "node", attributes)
    return node


def add_node(parent, child_text, existing_nodes, link=None, color=None, style=None):
    """
    générer un noeud pour le titre parent

    """
    for child in parent.findall("node"):
        if child.get("TEXT") == child_text:
            return child  # retourner les noeuds existants
    return create_node(parent, child_text, link=link, color=color, style=style)  # retourner les nouveaux noeuds


def excel_to_freemind(dataframe):
    """
    convertir en freemind Tree
    """
    root = ET.Element("map", {"version": "1.0.1"}) # c'est la version de freeMind
    root_values = dataframe.iloc[:, 0].dropna().unique()  # récupérer les valeurs de la 1er colonne dans l'excel pour faire un noeud de mère

    # créer des noeuds pour chaque valeur dans la 1er colonne (normalement on a les valeurs identiques dans 1a 1er colonne)
    # si on a des valeurs différentes dans la 1er colonne, les noeuds seront créés pour les différent valeurs
    for root_value in root_values:
        main_node = create_node(root, str(root_value), color="#0000FF", style="bubble")  # styler le root noeud

        # 
        filtered_rows = dataframe[dataframe.iloc[:, 0] == root_value]

        # ajouter les noeuds de fils pour chaque ligne 
        for _, row in filtered_rows.iterrows():
            current_parent = main_node
            existing_nodes = set()
            for col_value in row[1:]:  # procéder à partir de la 2eme colonne
                if pd.isna(col_value):
                    continue  # skip les valeurs vides
                url_candidate = str(col_value)
                is_url = url_candidate.startswith("http://") or url_candidate.startswith("https://")
                if is_url:
                    # vérifier si url exists. si oui, set url as current noeud
                    current_parent.set("LINK", url_candidate)  
                else:
                    current_parent = add_node(
                        current_parent,
                        str(col_value),
                        existing_nodes,
                        color="#FF4500", # set la couleur des noeuds fils
                        style="bezier"   # set le style des liens entre noeuds mère et noeuds fils
                    )
    return root


def save_freemind_xml(root, output_path):
    """sauvegarder freeMind tree en fichier .mm"""
    tree = ET.ElementTree(root)
    tree.write(output_path, encoding="utf-8", xml_declaration=True)



In [8]:

# input data provenant de l'excel
# file_path = r"C:\Users\jl0331el\Documents\python\side_project\myplantonline/exported_data_2.xlsx"
df = pd.read_excel(input_path)

# générer le freeMind tree en xml (.mm)
freemind_tree = excel_to_freemind(df)

# sauvegarder le fichier .mm
# output_path = r"C:\Users\jl0331el\OneDrive - EDF\XMind/output_data2.mm"
save_freemind_xml(freemind_tree, output_path)